In [1]:
!pip install moviepy
!pip install assemblyai
!pip install librosa
!pip install pydub
!pip install noisereduce
!pip install soundfile
!pip install huggingface_hub
!pip install transformer
!pip install evaluate
!pip install jiwer
!pip install sentencepiece
!pip install tensorflow_io
!pip install torchaudio
# !pip install tensorflow==2.16.1

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from moviepy.editor import *
import librosa
import numpy as np
import matplotlib.pyplot as plt
import os
import assemblyai as aai
import pandas as pd
from pydub import AudioSegment
from pydub.silence import split_on_silence
import noisereduce as nr
import soundfile as sf
import tensorflow as tf
import torchaudio

In [3]:
# print(tf.__version__)

In [4]:
def video_to_audio():

  folder_path = 'C:/Users/Harsh Patel/Desktop/Vintel/videos'
  for filename in os.listdir(folder_path):
      file_path = os.path.join(folder_path, filename)
      if os.path.isfile(file_path) and filename.endswith(('.mp4', '.mov', '.avi', '.mkv')):
            print(f"Processing video file: {filename}")

            # Open the video file and extract audio
            videoclip = VideoFileClip(file_path)
            audioclip = videoclip.audio

            # Write the audio file to mp3 format
            audio_output_path = f"C:/Users/Harsh Patel/Desktop/Vintel/audios/audiofile_{filename.split('.')[0]}.mp3"
            audioclip.write_audiofile(audio_output_path, codec="libmp3lame")
            print(f"Audio saved as: {audio_output_path}")


In [5]:
import os
from datasets import Dataset, DatasetDict
import soundfile as sf

def load_and_preprocess_data(audio_dir, transcript_dir):
    data = {'label': [], 'audio': [], 'transcript':[]}

    # Process audio files
    for index, audio_file in enumerate(os.listdir(audio_dir)):
        if audio_file.endswith('.mp3'):  # Adjust for audio file types
            file_path = os.path.join(audio_dir, audio_file)

            # Assuming preprocess_audio returns the processed audio and sample rate (sr)
#             processed_audio, sr = audio_reader(file_path)
            processed_audio, sample_rate = librosa.load(file_path, sr=16000, mono=True)

            # Append processed audio to the data dictionary
            data['audio'].append(processed_audio)

    # Process transcript files
    for index, transcript_file in enumerate(os.listdir(transcript_dir)):
        if transcript_file.endswith('.txt'):  # Adjust for transcript file types
            file_path = os.path.join(transcript_dir, transcript_file)

            # Read the transcript content and append it
            with open(file_path, 'r') as content:
                data['transcript'].append(content.read())

    # Ensure the label count matches the number of audios/transcripts
    num_entries = min(len(data['audio']), len(data['transcript']))  # Match the number of entries
    data['audio'] = data['audio'][:num_entries]  # Trim excess audio if necessary
    data['transcript'] = data['transcript'][:num_entries]  # Trim excess transcript if necessary

    data['label'] = [1] * num_entries  # Create a list of labels (e.g., all set to '1')

    return data

# Load and preprocess the audio and transcript data
audio_data1 = load_and_preprocess_data('audios', 'transcripts')

train_dict = {k: v[:8] for k, v in audio_data1.items()}
test_dict = {k: v[8:] for k, v in audio_data1.items()}


# Convert it into a Hugging Face dataset
dataset1 = DatasetDict({
    'train': Dataset.from_dict(train_dict),
    'test': Dataset.from_dict(test_dict)# Split later into train/test if needed
})

print(dataset1)


DatasetDict({
    train: Dataset({
        features: ['label', 'audio', 'transcript'],
        num_rows: 8
    })
    test: Dataset({
        features: ['label', 'audio', 'transcript'],
        num_rows: 3
    })
})


In [15]:
from transformers import Speech2TextProcessor, TFSpeech2TextForConditionalGeneration


processor_tf = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")
model_tf = TFSpeech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")

2024-09-08 17:33:51.420030: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
All PyTorch model weights were used when initializing TFSpeech2TextForConditionalGeneration.

All the weights of TFSpeech2TextForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFSpeech2TextForConditionalGeneration for predictions without further training.


In [16]:
# max_length = 1024  # Define a max length for the input sequence

input_features = processor_tf(
    dataset1['train']["audio"],
    sampling_rate=16000,
    return_tensors="tf",
    padding=True,  # Apply padding
    # max_length=max_length,  # Truncate to max_length
    # truncation=True  # Ensure truncation is applied
)

# Check input shape again
print(input_features['input_features'].shape)


input_labels = processor_tf.tokenizer(
    dataset1['train']["transcript"],
#     sampling_rate=16000,
    return_tensors="tf",
    padding=True,

    # max_length=max_length,  # Truncate to max_length
    # truncation=True  # Ensure truncation is applied
)

# Check input shape again
print(input_labels['input_ids'].shape)


Token indices sequence length is longer than the specified maximum sequence length for this model (2646 > 1024). Running this sequence through the model will result in indexing errors


(8, 170523, 80)
(8, 5452)


In [17]:
padded_inputs= np.array(input_features['input_features'], dtype= np.float32)
padded_labels = np.array(input_labels['input_ids'], dtype= np.float32)

print(padded_inputs.shape)  # Should output (num_samples, sequence_length, num_features)
print(padded_labels.shape)


(8, 170523, 80)
(8, 5452)


In [10]:
import jiwer

def wer_metric(y_true, y_pred):
    # Convert the tensors to text sequences
    y_true_text = processor_tf.batch_decode(y_true, skip_special_tokens=True)
    y_pred_text = processor_tf.batch_decode(y_pred, skip_special_tokens=True)

    # Calculate WER using jiwer
    wer_score = jiwer.wer(y_true_text, y_pred_text)

    return wer_score


In [11]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Split dataset into training and testing
xtrain, xtest, ytrain, ytest = train_test_split(padded_inputs, padded_labels, test_size=0.2, shuffle=False)

# Define the model
model_asr = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(None, 80)),  # Input shape: (sequence_length, features)

    # Apply 1D convolutions over the input sequence
    tf.keras.layers.Conv1D(256, 3, activation='relu'),
    tf.keras.layers.Conv1D(512, 3, activation='relu'),

    # MaxPooling layer
    tf.keras.layers.MaxPooling1D(),

    # Dropout to prevent overfitting
    tf.keras.layers.Dropout(0.3),

    # Use GlobalAveragePooling1D to remove the need for a fixed input length
    tf.keras.layers.GlobalAveragePooling1D(),

    # Fully connected layers
    # tf.keras.layers.Dense(128, activation='relu'),

    # Output layer (adjust number of units according to your task)
    tf.keras.layers.Dense(4091, activation='softmax')
])

# Print model summary
model_asr.summary()

# Define an optimizer and compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, None, 256)         61696     
                                                                 
 conv1d_1 (Conv1D)           (None, None, 512)         393728    
                                                                 
 max_pooling1d (MaxPooling1  (None, None, 512)         0         
 D)                                                              
                                                                 
 dropout_62 (Dropout)        (None, None, 512)         0         
                                                                 
 global_average_pooling1d (  (None, 512)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 4091)              2

In [12]:
model_tf.compile(optimizer= optimizer, loss= tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [13]:
len(xtrain[1])
ytrain[1].shape

(5452,)

In [ ]:
import tensorflow as tf

# Assuming `logits` is the output from your model
# `labels` is the true label sequence
# `input_length` is the length of the input sequence (before padding)
# `label_length` is the length of each label sequence



# model_asr.compile(optimizer=optimizer, loss= tf.keras.losses.CategoricalCrossentropy(from_logits=True),  metrics= 'accuracy')


# Train the model
# model_asr.fit(xtrain, ytrain, epochs=10, batch_size= 256)
# model_tf.fit(
#     x= input_features['input_features'],
#     y=input_labels['input_ids'],  # Pass labels here
#     epochs=10,
#     batch_size=64
# )

# generated_ids = model_tf.generate(xtrain)

# transcription = processor_tf.batch_decode(generated_ids)

strategy= tf.distribute.MirroredStrategy(devices=["CPU:0", "CPU:1","CPU:2", "CPU:3"], cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
with strategy.scope():


  processor_tf = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")
  model_tf = TFSpeech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
  model_tf.compile(optimizer= optimizer, loss= tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

input_features = tf.expand_dims(xtrain[0], axis=0)  # Adds a batch dimension
labels = tf.expand_dims(ytrain[0], axis=0)  # Adds a batch dimension

model_tf(input_features= input_features, labels= labels, training= True)

In [ ]:
y_pred = model_asr.predict(xtest)
len(y_pred)
y_pred

### Pytorch Implementation

In [6]:
import os
import torch
import torchaudio
from torch.utils.data import Dataset
from transformers import Wav2Vec2Tokenizer

class SpeechDataset(Dataset):
    def __init__(self, audio_files, transcripts, tokenizer, feature_extractor):
        self.audio_files = audio_files
        self.transcripts = transcripts
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio_file = self.audio_files[idx]
        transcript = self.transcripts[idx]

        if not isinstance(transcript, str):
           
             raise TypeError(f"Expected transcript to be a string, got {type(transcript)}")


        # Load and preprocess audio
        audio_input, _ = torchaudio.load(audio_file)
        audio_input = self.feature_extractor(audio_input.squeeze().numpy(), sampling_rate=16000, return_tensors="pt").input_values.squeeze()
        audio_input = torch.tensor(audio_input, dtype=torch.float32)
        
        # Tokenize transcript
        target = self.tokenizer(transcript, return_tensors="pt").input_ids.squeeze()
        target = torch.tensor(target, dtype=torch.float32) 
        
        return {
            'input_values': audio_input,
            'labels': target
        }


In [44]:
import torch
from torch.utils.data import DataLoader
from transformers import Wav2Vec2ForCTC, Wav2Vec2FeatureExtractor, Wav2Vec2Tokenizer, AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC
from transformers import AdamW

# Initialize the tokenizer and feature extractor
# tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
# feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = AutoTokenizer.from_pretrained("facebook/wav2vec2-base-960h")
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")


# Load your dataset
audio_files = ["audios/audio_1.mp3"]  # List of paths to your audio files
transcripts = ["transcripts/transcript_1.txt"]  # Corresponding transcripts

dataset = SpeechDataset(audio_files, transcripts, tokenizer, feature_extractor)
# torch.
# Create DataLoader with multiprocessing
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)  # Adjust num_workers based on your CPU

# Initialize model


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

In [46]:
# model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h", torch_dtype= torch.float32)
model.train()

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(3):  # Adjust number of epochs
    for batch in dataloader:
        input_values = batch['input_values']
        labels = batch['labels']

        # Forward pass
        outputs = model(input_values=input_values, labels=labels)
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch}, Loss: {loss.item()}")


RuntimeError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [1, 1, 2, 32967837]

20

In [7]:
import torch
from torch.utils.data import Dataset
from transformers import AutoModelForCTC, AutoTokenizer, AutoFeatureExtractor
import numpy as np
import librosa
import multiprocessing
from multiprocessing import Pool, cpu_count

# Hyperparameters
hyperparams = {
    'epochs': 3,
    'batch_size': 512,
    'learning_rate': 0.001,
    'num_workers': cpu_count(),
    'sampling_rate': 16000  # Define the sampling rate
}

# Load model and tokenizer
model_name = "facebook/wav2vec2-base-960h"
model = AutoModelForCTC.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

# Dummy dataset class
class SpeechDataset(Dataset):
    def __init__(self, audio_files, transcripts, sampling_rate):
        self.audio_files = audio_files
        self.transcripts = transcripts
        self.feature_extractor = feature_extractor
        self.tokenizer = tokenizer
        self.sampling_rate = sampling_rate

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio_file = self.audio_files[idx]
        transcript = self.transcripts[idx]

        # Load and process audio
        audio_input, _ = librosa.load(audio_file, sr=self.sampling_rate)
        audio_input = self.feature_extractor(audio_input, return_tensors="pt", sampling_rate=self.sampling_rate).input_values.squeeze()

        # Tokenize transcript
        target = self.tokenizer(transcript, return_tensors="pt").input_ids.squeeze()

        return {'input_values': audio_input, 'labels': target}

# Data preprocessing function
def preprocess_data(idx, audio_files, transcripts, sampling_rate):
    dataset = SpeechDataset(audio_files, transcripts, sampling_rate)
    return dataset[idx]

# Example file paths and transcripts
audio_files = ["audios/audio_1.mp3"]  # List of paths to your audio files
transcripts = ["transcripts/transcript_1.txt"]  # Corresponding transcripts

# Create dataset
dataset = SpeechDataset(audio_files, transcripts, hyperparams['sampling_rate'])

# Multiprocessing for data loading
def load_data_in_parallel(audio_files, transcripts, sampling_rate):
    with Pool(2) as pool:
        results = pool.starmap(preprocess_data, [(i, audio_files, transcripts, sampling_rate) for i in range(len(audio_files))])
    return results

# Load data
data = load_data_in_parallel(audio_files, transcripts, hyperparams['sampling_rate'])

# Training loop
device = torch.device("gpu")
model.to(device)
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=hyperparams['learning_rate'])

for epoch in range(hyperparams['epochs']):
    for item in data:
        input_values = item['input_values'].unsqueeze(0).to(device)  # Add batch dimension
        labels = item['labels'].unsqueeze(0).to(device)  # Add batch dimension

        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_values, labels=labels)
        loss = outputs.loss

        # Backward pass
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

print("Training complete!")


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

KeyboardInterrupt: 

In [ ]:
import torch
from transformers import AutoModelForCTC, AutoTokenizer, AutoFeatureExtractor
import librosa
import os
from multiprocessing import Pool, cpu_count
import numpy as np

# Hyperparameters
hyperparams = {
    'epochs': 3,
    'batch_size': 170000,  # Adjust batch size according to memory constraints
    'learning_rate': 0.001,
    'num_workers': 4,
    'sampling_rate': 16000  # Define the sampling rate
}

# Load model and tokenizer
model_name = "facebook/wav2vec2-base-960h"
model = AutoModelForCTC.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

class SpeechDataset:
    def __init__(self, audio_files, transcripts, sampling_rate):
        self.audio_files = audio_files
        self.transcripts = transcripts
        self.sampling_rate = sampling_rate

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio_file = self.audio_files[idx]
        transcript = self.transcripts[idx]

        # Load and process audio
        audio_input, _ = librosa.load(audio_file, sr=self.sampling_rate)
        audio_input = feature_extractor(audio_input, return_tensors="pt", sampling_rate=self.sampling_rate).input_values.squeeze()

        # Tokenize transcript
        target = tokenizer(transcript, return_tensors="pt").input_ids.squeeze()

        return {'input_values': audio_input.numpy(), 'labels': target.numpy()}

# Data preprocessing function
def preprocess_data(index, audio_files, transcripts, sampling_rate):
    dataset = SpeechDataset(audio_files, transcripts, sampling_rate)
    return dataset[index]

# Load example file paths and transcripts
audio_files = ["audios/audio_1.mp3"]  # List of paths to your audio files
transcripts = ["transcripts/transcript_1.txt"]  # Corresponding transcripts

# Create dataset
dataset = SpeechDataset(audio_files, transcripts, hyperparams['sampling_rate'])

# Function to load data in parallel
def load_data_in_parallel(audio_files, transcripts, sampling_rate, batch_size):
    with Pool(hyperparams['num_workers']) as pool:
        indices = list(range(len(audio_files)))
        results = pool.starmap(preprocess_data, [(i, audio_files, transcripts, sampling_rate) for i in indices])
    
    # Group results into batches
    batches = [results[i:i + batch_size] for i in range(0, len(results), batch_size)]
    return batches

# Load data
data_batches = load_data_in_parallel(audio_files, transcripts, hyperparams['sampling_rate'], hyperparams['batch_size'])

# Training loop
device = torch.device("cpu")
model.to(device)
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=hyperparams['learning_rate'])

for epoch in range(hyperparams['epochs']):
    total_loss = 0
    for batch in data_batches:
        input_values = torch.tensor([item['input_values'] for item in batch]).to(device)
        labels = torch.tensor([item['labels'] for item in batch]).to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_values, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Average Loss: {total_loss / len(data_batches)}")

print("Training complete!")


In [11]:
!pip install omegaconf

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 53.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=610094fb227983d64502d34eacebaf8fa372fa59ca399658638140dd595ed903
  Stored in directory: /home/jovyan/.cache/pip/wheels/b1/a3/c2/6df046c09459b73cc9bb6c4401b0be6c47048baf9a1617c485
Successfully built antlr4-python3-runtime

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import torch
import zipfile
import torchaudio
from glob import glob

device = torch.device('cpu')  # gpu also works, but our models are fast enough for CPU
model, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                       model='silero_stt',
                                       language='en', # also available 'de', 'es'
                                       device=device)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils  # see function signature for details

# download a single file in any format compatible with TorchAudio
# torch.hub.download_url_to_file('https://opus-codec.org/static/examples/samples/speech_orig.wav',
#                                dst ='speech_orig.wav', progress=True)
test_files = glob('audios/audio_1.mp3')
batches = split_into_batches(test_files, batch_size=3)
input = prepare_model_input(read_batch(batches[0]),
                            device=device)

output = model(input)
for example in output:
    print(decoder(example.cpu()))

Using cache found in /home/jovyan/.cache/torch/hub/snakers4_silero-models_master
